# Submit NICS calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import cdxml, computational_resources, editors, inputs

In [ ]:
GaussianNicsWorkChain = plugins.WorkflowFactory("nanotech_empa.gaussian.nics")

In [ ]:
# Structure selector.
structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        cdxml.CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors=[
        awb.BasicStructureEditor(title="Edit structure"),
    ],
    storable=True,
    node_class="StructureData",
)

display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="Gaussian code:", default_calc_job_plugin="gaussian"
)
resources = computational_resources.ProcessResourcesWidget()

output = ipw.Output()

In [ ]:
# Functional.
functional = ipw.Dropdown(
    value="B3LYP",
    options=[
        "B3LYP",
        "PBE",
    ],
    description="Functional:",
    style={"description_width": "initial"},
)
# Basis set.
basis = ipw.Dropdown(
    value="6-311G(d,p)",
    options=[
        "6-311G(d,p)",
    ],
    description="Basis set:",
    style={"description_width": "initial"},
)
# Multiplicity.
multiplicity = ipw.IntText(
    value=0,
    description="Multiplicity:",
    style={"description_width": "initial"},
)
# Charge.
charge = ipw.IntText(
    value=0,
    description="Charge:",
    style={"description_width": "initial"},
)
# VdW.
vdw = ipw.Dropdown(
    value="None",
    options=[
        "None",
        "GD3",
        "GD3BJ",
    ],
    description="VdW correction:",
    style={"description_width": "initial"},
)
# Height of NICS centers.
height = ipw.FloatText(
    value=1.0,
    description="Height of NICS centers (Angstrom):",
    style={"description_width": "initial"},
)
# Optimization.
opt = ipw.Checkbox(
    value=True,
    description="Optimize",
    style={"description_width": "initial"},
)

In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
def prepare_geometry_optimization():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select Gaussian code."
    else:
        can_submit, msg = True, "Submitted NICS workchain"

    if not can_submit:
        with output:
            print(msg)
            return

    builder = GaussianNicsWorkChain.get_builder()
    builder.metadata.label = "Gaussian_NICS"
    builder.metadata.description = workflow_description.value
    builder.gaussian_code = orm.load_code(code_input_widget.value)
    builder.structure = structure_selector.structure_node
    builder.multiplicity = orm.Int(multiplicity.value)
    builder.charge = orm.Int(charge.value)
    builder.functional = orm.Str(functional.value)
    builder.basis_set = orm.Str(basis.value)
    builder.empirical_dispersion = orm.Str(vdw.value)
    builder.opt = orm.Bool(opt.value)

    builder.options = orm.Dict(
        {
            "max_wallclock_seconds": resources.walltime_seconds,
            "max_memory_kb": resources.total_memory_gb * 1024 * 1024,
            "resources": {
                "num_machines": resources.nodes,
                "tot_num_mpiprocs": resources.tasks_per_node,
            },
        }
    )

    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    GaussianNicsWorkChain,
    inputs_generator=prepare_geometry_optimization,
    disable_after_submit=False,
    append_output=True,
)

## Parameters

In [ ]:
display(ipw.VBox([functional, basis, multiplicity, charge, vdw, height, opt]))

# Code and resources

In [ ]:
display(
    code_input_widget,
    resources,
)

# Submit

In [ ]:
display(workflow_description, btn_submit, output)